<a href="https://colab.research.google.com/github/srivatsan88/YouTubeLI/blob/master/Tensorflow_Keras_Tuner_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!nvidia-smi

Sun Apr 26 00:08:17 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
!pip install -U keras-tuner

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import os
import datetime
import tensorflow_datasets as tfds
from kerastuner.engine.hyperparameters import HyperParameters

%load_ext tensorboard

In [0]:
tf.config.experimental.list_physical_devices()

In [0]:
import tensorflow_datasets as tfds
datasets, info = tfds.load(name='fashion_mnist', with_info=True, as_supervised=True, try_gcs=True, split=['train','test'])

In [0]:
print(info.features)
print(info.features["label"].num_classes)
print(info.features["label"].names)

In [0]:
fm_train, fm_test = datasets[0], datasets[1]
fm_val = fm_test.take(3000)
fm_test = fm_test.skip(3000)

In [0]:
import matplotlib.pyplot as plt
import numpy as np
for fm_sample in fm_train.take(5):  
  image, label = fm_sample[0], fm_sample[1]

  plt.figure()
  plt.imshow(image.numpy()[:, :, 0].astype(np.float32), cmap=plt.get_cmap("gray"))
  plt.show()
  print("Label: %d" % label.numpy())
  print("Category: %s" % info.features["label"].names[label.numpy()])

In [0]:
def scale(image, label):
  image = tf.cast(image, tf.float32)
  image /= 255.0

  return image, label

In [0]:
def get_dataset(batch_size=64):
  train_dataset_scaled = fm_train.map(scale).shuffle(60000).batch(batch_size)
  test_dataset_scaled = fm_test.map(scale).batch(batch_size)
  val_dataset_scaled = fm_val.map(scale).batch(batch_size)
  return train_dataset_scaled, test_dataset_scaled, val_dataset_scaled

In [0]:
hp = HyperParameters()
hp.Choice('learning_rate', [1e-1, 1e-3])
hp.Int('conv_blocks', 3, 4, default=3)
hp.Int('hidden_size', 128, 256, step=64, default=128)

In [0]:
def build_model(hp):
    inputs = tf.keras.Input(shape=(28, 28, 1))
    x = inputs
    for i in range(hp.get('conv_blocks')):
        filters = hp.Int('filters_' + str(i), 32, 256, step=64)
        for _ in range(2):
            x = tf.keras.layers.Convolution2D(
              filters, kernel_size=(3, 3), padding='same')(x)
            x = tf.keras.layers.BatchNormalization()(x)
            x = tf.keras.layers.ReLU()(x)
        if hp.Choice('pooling_' + str(i), ['avg', 'max']) == 'max':
            x = tf.keras.layers.MaxPool2D()(x)
        else:
            x = tf.keras.layers.AvgPool2D()(x)
    x = tf.keras.layers.GlobalAvgPool2D()(x)
    x = tf.keras.layers.Dense( hp.get('hidden_size'),activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    outputs = tf.keras.layers.Dense(10, activation='softmax')(x)

    model = tf.keras.Model(inputs, outputs)
    model.compile(
      optimizer=tf.keras.optimizers.Adam(hp.get('learning_rate')),
      loss='sparse_categorical_crossentropy',
      metrics=['accuracy'])
    return model

In [0]:
import kerastuner as kt

tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    hyperparameters=hp,
    max_epochs=5,
    hyperband_iterations=2)
  

In [0]:
tuner.search_space_summary()


In [0]:
train_dataset, test_dataset, val_dataset = get_dataset()
train_dataset.cache()
val_dataset.cache()

tuner.search(train_dataset,
             validation_data=val_dataset,
             epochs=5,
             callbacks=[tf.keras.callbacks.EarlyStopping(patience=1)])

In [0]:
best_model = tuner.get_best_models(1)[0]

In [0]:
best_model.summary()

In [0]:
best_hyperparameters = tuner.get_best_hyperparameters(1)[0]


In [0]:
best_hyperparameters.values

In [0]:
tuner.results_summary()

In [0]:
best_model.save('/tmp/fashion.hdf5')

In [0]:
 !nvidia-smi

In [0]:
!ls -alrt /tmp/fashion.hdf5

In [0]:
tf.keras.backend.clear_session()
fminst_load = tf.keras.models.load_model('/tmp/fashion.hdf5',compile = False)

In [0]:
for test_sample in fm_test.take(10):  
  image, label = test_sample[0], test_sample[1]

  img = tf.keras.preprocessing.image.img_to_array(image)                    
  img = np.expand_dims(img, axis=0)         
  img /= 255.    
  pred=fminst_load.predict(img)
  plt.figure()
  plt.imshow(image.numpy()[:, :, 0].astype(np.float32), cmap=plt.get_cmap("gray"))
  plt.show()
  print("Actual Label: %s" % info.features["label"].names[label.numpy()])
  print("Predicted Label: %s" % info.features["label"].names[np.argmax(pred)])


In [0]:
_, accuracy = best_model.evaluate(test_dataset)

In [0]:
accuracy